# Introduction to Python: Data handling and data cleaning

### General elements to consider *before* getting started 

Builds on the assumption that you already have defined the overall goal of the analysis and that required data is already known.

1) Do I have the infrastructure to store and process the data? (local machine vs. computing cluster vs. cloud)

2) If handling personal or sensitive information, have the data subjects consented to your planned analysis?

3) Does your planned analysis comply with the Terms of Services of the data owner?

4) Do I have a version control system in place to track processing steps? (Data provenance)

### General elements to consider *when* getting started

1) Is data integrity ensured when loading the data? Is data loaded in the correct fomat? (categoricals, dates etc.)

2) Is missing data present? (drop vs. impute)

3) Are outliers present? Are they informative?

4) Are duplicate values present? Are they informative?

5) Are the processing steps well documented enough for others to understand?

**==> Explore and visualize before getting into modelling!**

# Research question

Are the recidivism predictions from the COMPAS software fair? Specifically, are the Type I & Type II errors from the COMPAS prediction of the risk of recidivism similar across subgroups of the population, e.g. defined by gender or race?

**Type I**: Wrongly accusing the innocent (*False Positive*).

**Type II**: Letting the guilty go free (*False Negative*).

# Let's start

Some information about the dataset:

- Data source: Sheriff’s Office of Broward County, Florida, USA
- Period of observation: Two full years, 2013-14
- Unit of observation: Assessment per defendant per case.

### Import Libraries
Import relevant libraries first. Unless there is a reason not to do so, always add libraries at the beginning of the script as it allows you to keep better track of the libraries used.

In general, KISS holds: **K**eep **I**t **S**imple, **S**tupid!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from helper_functions import filter_unique

### Import Private Functions
As ideally you have separately saved the functions that you have defined yourself for other work before, this is the place to import them.

In [ ]:
%run my_functions.ipynb

----> Naming conventions

### Import Data

Here, we are interested in the [COMPAS recidivism dataset](https://github.com/propublica/compas-analysis/blob/master/compas-scores-raw.csv). We can either download it and load it from local or directly load it from the source.

In [ ]:
!cat compas-scores-raw.csv

In [ ]:
df_raw = pd.read_csv('compas-scores-raw.csv', dtype={'Person_ID': str})

----> Data types

In [ ]:
print(df_raw)

Sometimes it makes sense to cast certain columns directly to a certain data type as otherwise information will be lost. You can do that via the `dtype` argument, e.g.: dtype={'Person_ID': str}

In [ ]:
type(df_raw)

### Basic exploration

Let's first look at the size of the table

In [ ]:
dir(df_raw)

In [ ]:
df_raw.shape

Which data types are used and are missing values present?

In [ ]:
df_raw.info()

Why do we have a few missings in the ScoreText variable? But first, let's look how the first three records look like:

In [ ]:
df_raw.head(3) #df_raw.tail(3)

Is the whole dataset all about Kevin Fisher?

In [ ]:
df_raw.head(10)

No, but apparently there are three observations for each Person/Case. Is that the case for every entry?

In [ ]:
df_raw['RawScore'].min()

In [ ]:
df_raw['RawScore'].describe()

In [ ]:
(df_raw.groupby(['Person_ID', 'AssessmentID', 'Case_ID'])['LastName']
  .count()
  .reset_index()
  .describe())

In [ ]:
(df_raw
 .groupby(['Person_ID', 'AssessmentID', 'Case_ID'])['LastName']
 .count()
.reset_index()
.describe())

Yes, apparently that's the case for every Person/Case. If they are perfect duplicates, we may remove them.

In [ ]:
df_raw.duplicated()

In [ ]:
df_raw.duplicated().sum()

OK, they differ in some fields. Which though?

In [ ]:
df_raw[0:3]

In [ ]:
df_raw[0:3].nunique()

Is that similar for all Persons/Cases?

In [ ]:
df_raw.groupby(['Person_ID', 'AssessmentID', 'Case_ID']).nunique().describe().transpose()

In [ ]:
df_raw.groupby(['Person_ID', 'AssessmentID', 'Case_ID']).nunique().describe().transpose()

What's the Scale_ID? A bit hidden in the methodology note, it says: *Each pretrial defendant received at least three COMPAS scores: “Risk of Recidivism,” “Risk of Violence” and “Risk of Failure to Appear.”*. This should be stated in `DisplayText` then probably.

In [ ]:
df_raw.DisplayText.unique()

OK, we can now be quite certain that there are no duplicates within a given `Person_ID` + `AssessmentID` + `Case_ID` combination and that those combinations are also unique in the dataset. But can we be sure no incident is accidently recorded twice with a different `Case_ID`, for example? In the lecture, we trust in the data quality, in the exercises, however, you don't, so be excited!

### Reshaping data

Right now, for a given `Person_ID` + `AssessmentID` + `Case_ID` combination, 23 out of the 28 columns are duplicates. Can we reduce the size and thus the memory space? We could pivot the table from long to wide. We would go from 60843x28 to 20421x33. Let's try it first on a small subset.

In [ ]:
df_raw[0:3]

In [ ]:
df_raw.loc[0:2, ['Case_ID', 'LastName']]

In [ ]:
test = (df_raw
 .loc[0:2,['Case_ID',
           'DisplayText',
           'RawScore']]
 .pivot_table(index='Case_ID',
              columns='DisplayText',
              values='RawScore')
 .reset_index())

In [ ]:
test['this is a column']

In [ ]:
test.this a column

In [ ]:
test

In [ ]:
test['Risk of Failure to Appear']

OK, that seems to work, but the new column names don't match good practices. Let's remove the spaces in the column `DisplayText`.

In [ ]:
df_raw['DisplayText'] = df_raw.DisplayText.str.replace(' ', '')

In [ ]:
df_raw.DisplayText.unique()

We now pivot the whole dataset.

In [ ]:
pivot_fields = ['DisplayText', 'RawScore', 'DecileScore', 'ScoreText']
pivot_index = [col for col in df_raw.columns.tolist() if col not in pivot_fields]

In [ ]:
pivot_index

----> 05_Control_Logics
----> 07_Comprehensions_and_Functional_Programming

We still have `Scale_ID` in it (which varies within a group) and the `MiddleName` (which in many cases are missing). We could handle both to be accounted for when pivoting, but for now, we drop them.

In [ ]:
pivot_index.remove('MiddleName')
pivot_index.remove('Scale_ID')

In [ ]:
df_wide = df_raw.pivot_table(index=pivot_index, columns='DisplayText', values=['RawScore', 'DecileScore', 'ScoreText'], aggfunc='first')

In [ ]:
df_wide.info()

This looks good. However, we have a multi-level index in the columns right now. Let's reduce it and separate with an underscore.

In [ ]:
df_wide.columns = ['{}_{}'.format(col[1], col[0]) for col in df_wide.columns]

We put the index back as a normal column.

In [ ]:
df_wide.reset_index(inplace=True)

In [ ]:
df_wide

Check whether we have some duplicates in here.

In [ ]:
df_wide[['Person_ID', 'AssessmentID', 'Case_ID']].duplicated().sum()

Were some people assessed multiple times?

In [ ]:
df_wide.groupby('Person_ID')['Case_ID'].count().describe()

### Simple validity checks

Let's assume we are interested in the **risk of recidivism** disaggregated by **gender** and **race**. We now explore whether there is some obvious artifacts in the data.

##### Recidivism

In [ ]:
df_wide.loc[:,df_wide.columns.str.contains("RiskofRecidivism")].describe()

One variable is missing, since it is an object type.

In [ ]:
df_wide.RiskofRecidivism_DecileScore.value_counts()

Why is there a -1 in the `RiskofRecidivism_DecileScore`? In the [methodology](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm) it says: *COMPAS scores for each defendant ranged from 1 to 10, with ten being the highest risk. Scores 1 to 4 were labeled by COMPAS as “Low”; 5 to 7 were labeled “Medium”; and 8 to 10 were labeled “High.”*. Let's check the number of classes in `RiskofRecidivism_DecileScore`.

In [ ]:
df_wide.RiskofRecidivism_DecileScore.unique()

There is no 0, but a -1. Let's have a look at the cases.

In [ ]:
df_wide[df_wide.RiskofRecidivism_DecileScore == -1]

These are exactly those 36 rows that make up for the 36 missing values in the field `RiskofRecidivism_ScoreText`. Interestingly, they all have a `RiskofRecidivism_RawScore` of -1.0. Does it hold true for all cases with a -1.0 score?

In [ ]:
df_wide[df_wide.RiskofRecidivism_RawScore == -1]['RiskofRecidivism_ScoreText'].unique()

No. But maybe some other variables causes it. Let's look for those variables that have only one value across our cases.

In [ ]:
unique_counts = df_wide[df_wide.RiskofRecidivism_DecileScore == -1].nunique()

In [ ]:
unique_counts

Let's see if any of those variables that don't vary here may only take this value across these cases .

In [ ]:
df_wide.loc[:,unique_counts[unique_counts <= 1].index]['RiskofRecidivism_DecileScore'].unique()

No, there is no direct link visible. We could further explore that, e.g. by checking whether self-constructed deciles on the `RawScore` match the `DecileScore` and run other checks. For simplicity, we simply drop them here.

In [ ]:
df_wide = df_wide.loc[df_wide.RiskofRecidivism_DecileScore != -1].copy()

In [ ]:
df_wide.RiskofRecidivism_DecileScore.value_counts()

In [ ]:
plt.hist(df_wide.RiskofRecidivism_DecileScore, density = True)
plt.show()

##### Gender

In [ ]:
df_wide.Sex_Code_Text.value_counts()

Get share of women

In [ ]:
df_wide.loc[df_wide.Sex_Code_Text == 'Female', 'Sex_Code_Text'].count() / df_wide.loc[:,'Sex_Code_Text'].count()

Variable seems ok. Could do sanity check with external sources.

##### Race

In [ ]:
df_wide.Ethnic_Code_Text.value_counts()

We see that there are two entries for African-Am(erican). Let's align them.

In [ ]:
df_wide.loc[df_wide.Ethnic_Code_Text == 'African-Am', 'Ethnic_Code_Text'] = 'African-American'

Look at them as a table:

In [ ]:
df_wide.groupby(['Ethnic_Code_Text', 'Sex_Code_Text']).size().unstack().reset_index()

One could further explore the difference between Arabic and Oriental and understand which people are grouped in others. We don't do that here. We simply plot it.

By race

In [ ]:
plt.hist(df_wide.loc[df_wide.Ethnic_Code_Text == 'African-American', 'RiskofRecidivism_DecileScore'], density=True)

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,5))

# African-Americans
axs[0].hist(df_wide.loc[df_wide.Ethnic_Code_Text == 'African-American', 'RiskofRecidivism_DecileScore'], density=True)
axs[1].hist(df_wide.loc[df_wide.Ethnic_Code_Text == 'Caucasian', 'RiskofRecidivism_DecileScore'], density=True)
axs[2].hist(df_wide.loc[df_wide.Ethnic_Code_Text == 'Hispanic', 'RiskofRecidivism_DecileScore'], density=True)

axs[0].set_xlabel('Risk of Recidivism - Decile Score')
axs[1].set_xlabel('Risk of Recidivism - Decile Score')
axs[2].set_xlabel('Risk of Recidivism - Decile Score')

axs[0].set_ylabel('Rel. Frequency')
axs[1].set_ylabel('Rel. Frequency')
axs[2].set_ylabel('Rel. Frequency')

axs[0].set_title('Histogram of Risk of Recidivism (African-Americans only)')
axs[1].set_title('Histogram of Risk of Recidivism (Caucasians only)')
axs[2].set_title('Histogram of Risk of Recidivism (Hispanics only)')

plt.show()

By gender

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (18,5))

# African-Americans
axs[0].hist(df_wide.loc[df_wide.Sex_Code_Text == 'Male', 'RiskofRecidivism_DecileScore'], density=True)
axs[1].hist(df_wide.loc[df_wide.Sex_Code_Text == 'Female', 'RiskofRecidivism_DecileScore'], density=True)

axs[0].set_xlabel('Risk of Recidivism - Decile Score')
axs[1].set_xlabel('Risk of Recidivism - Decile Score')

axs[0].set_ylabel('Rel. Frequency')
axs[1].set_ylabel('Rel. Frequency')

axs[0].set_title('Histogram of Risk of Recidivism (Men only)')
axs[1].set_title('Histogram of Risk of Recidivism (Women only)')

plt.show()

Especially for race, we see quite some big differences. Can we use the existing dataset to check whether those COMPAS predictions are fair?

### Initial check for bias

Recall that we have some people multiple times in the dataset. Following the histograms above, we should see a comparatively higher share of African-Americans in this group.

##### Type II error: Let the guilty go free, i.e. reoffended although being judged low risk.

In [ ]:
df_multiple = df_wide[df_wide.Person_ID.duplicated(keep=False)]

In [ ]:
len(df_multiple.Person_ID.unique())

In [ ]:
df_multiple.Person_ID.nunique()

In [ ]:
df_multiple.groupby('Person_ID')['Screening_Date'].nunique().describe()

We see some individuals were screened multiple times on the same day. One should explore that if there is a relevant reason for that. Here we simply drop them.

In [ ]:
df_multiple.groupby('Person_ID')['Screening_Date'].nunique() != 1

In [ ]:
list(set(list(a)))

In [ ]:
def filter_unique(group):
    return group['Screening_Date'].nunique() == len(group['Screening_Date'])

In [ ]:
df_multiple = (df_multiple
               .groupby('Person_ID')
               .filter(filter_unique)
               .reset_index(drop = True)
               .copy())

In [ ]:
df_multiple = (
    df_multiple
    .groupby('Person_ID')
    .filter(lambda group: group['Screening_Date'].nunique() == len(group['Screening_Date']))
    .reset_index(drop=True)
    .copy()
)

Let's try if that has worked:

In [ ]:
df_multiple.groupby('Person_ID')['Screening_Date'].nunique().describe()

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,5))

# African-Americans
axs[0].hist(df_multiple.loc[df_multiple.Ethnic_Code_Text == 'African-American', 'RiskofRecidivism_DecileScore'], density=True)
axs[1].hist(df_multiple.loc[df_multiple.Ethnic_Code_Text == 'Caucasian', 'RiskofRecidivism_DecileScore'], density=True)
axs[2].hist(df_multiple.loc[df_multiple.Ethnic_Code_Text == 'Hispanic', 'RiskofRecidivism_DecileScore'], density=True)

axs[0].set_xlabel('Risk of Recidivism - Decile Score')
axs[1].set_xlabel('Risk of Recidivism - Decile Score')
axs[2].set_xlabel('Risk of Recidivism - Decile Score')

axs[0].set_ylabel('Rel. Frequency')
axs[1].set_ylabel('Rel. Frequency')
axs[2].set_ylabel('Rel. Frequency')

axs[0].set_title('Histogram of Risk of Recidivism (African-Americans only)')
axs[1].set_title('Histogram of Risk of Recidivism (Caucasians only)')
axs[2].set_title('Histogram of Risk of Recidivism (Hispanics only)')

plt.show()

In [ ]:
df_recid = df_multiple.groupby(['Ethnic_Code_Text', 'RiskofRecidivism_ScoreText']).size().unstack().reset_index()

In [ ]:
df_recid['Total'] = df_recid['Low'] + df_recid['Medium'] + df_recid['High']
df_recid['low_share'] = df_recid['Low']/df_recid['Total']
df_recid

We see that only about 26% percent of the African-Americans who apparently reoffended were assumed to be low-risk of recidivism, whereas for the Caucasians and Hispanics, these this share was around 60%. One may conclude from that that Caucasians and Hispanics are more likely perceived to be low risk even though they aren't vis-à-vis African-Americans.

Let's repeat that the other way around.

##### Type I: Wrongfully accuse the innocent, i.e. hasn't reoffended although being judged high risk.

In [ ]:
df_single = df_wide[~df_wide.Person_ID.duplicated(keep=False)]

In [ ]:
len(df_single.Person_ID)

In [ ]:
df_single.Person_ID.nunique()

In [ ]:
df_norecid = df_single.groupby(['Ethnic_Code_Text', 'RiskofRecidivism_ScoreText']).size().unstack().reset_index()

In [ ]:
df_norecid['Total'] = df_norecid['Low'] + df_norecid['Medium'] + df_norecid['High']
df_norecid['high_share'] = df_norecid['High']/df_norecid['Total']
df_norecid

We see that about 24% percent of the African-Americans who have not (yet) reoffended were assumed to be high-risk of recidivism, whereas for the Caucasians and Hispanics, this share was below 10%. One may conclude from that that African-Americans are more likely perceived to be high risk even though they may not be vis-à-vis African-Americans.

What else could we do to assess the fairness of the COMPAS prediction?